In [ ]:
# |default_exp whisper

# Constantes

In [ ]:
# |export

AUDIO_PATH = "audios"

# list mp3 files

In [ ]:
# |export

from mongo_episode import get_audio_path
import os, glob


def list_mp3_files(audio_path=AUDIO_PATH):
    fullpath = get_audio_path(audio_path, year="")

    return glob.glob(os.path.join(fullpath, "**/*.mp3"), recursive=True)

In [ ]:
list_mp3_files()[:5]

['/mnt/c/Users/f279814/git/lmelp/audios/2016/14007-13.11.2016-ITEMA_21134385-0.mp3']

# extract whisper

In [ ]:
# |export

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


def extract_whisper(mp3_filename):

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    model_id = "openai/whisper-large-v3-turbo"

    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    )
    model.to(device)

    processor = AutoProcessor.from_pretrained(model_id)

    generate_kwargs = {
        "language": "french",
    }

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype,
        device=device,
        chunk_length_s=30,
        batch_size=16,  # batch size for inference - set based on your device
        generate_kwargs=generate_kwargs,
    )

    dataset = load_dataset(
        "distil-whisper/librispeech_long", "clean", split="validation"
    )
    sample = dataset[0]["audio"]

    result = pipe(
        mp3_filename,
        return_timestamps=True,
    )

    return result["text"]

In [ ]:
mp3_1 = list_mp3_files()[0]

whisper = extract_whisper(mp3_1)

whisper[:150]

Device set to use cuda:0
/home/guillaume/miniconda/envs/whisper/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=french, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=french.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


' Musique Le masque et la plume Musique Bonsoir à tous, bienvenue en public au studio Sacha Guitry de la maison de Radio France pour un masqué la plume'

# store whisper in db 

In [ ]:
# |export

from bson import ObjectId


def store_whisper_in_db(whisper, collection, oid, force=False, verbose=False):
    """
    whisper: str, la transcription du fichier audio
    collection: pymongo collection
    oid: str, l'identifiant de l episode
    force: bool, si True, on ecrase le whisper existant

    return True si le whisper a ete stocke, False sinon
    """

    # Récupération du document
    document_entry = collection.find_one({"_id": ObjectId(oid)})

    if document_entry is None:
        if verbose:
            print(f"Document avec l'oid {oid} non trouvé")
        return False

    if "whisper" in document_entry and not force:
        if verbose:
            print(
                f"Whisper déjà stocké pour l'oid {oid}m et on ne force pas le stockage"
            )
        return False
    else:
        document_entry["whisper"] = whisper
        collection.update_one({"_id": ObjectId(oid)}, {"$set": document_entry})
        if verbose:
            print(f"Whisper stocké pour l'oid {oid}")
        return True

In [ ]:
from connection import get_collection

col = get_collection()
oid = "6773e32258fc5717f3516b98"
store_whisper_in_db("test whisper", col, oid, force=True, verbose=True)

Whisper stocké pour l'oid 6773e32258fc5717f3516b98


True

# extract py

In [ ]:
from nbdev.export import nb_export

nb_export("09 whisper mp3.ipynb", ".")